In [ ]:
from itertools import pairwise

import arviz as az
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import numpyro
import numpyro.distributions as dist
import optax
from flax import nnx
from jax import random
from jaxtyping import Array, Float, Int
from numpyro.contrib.module import nnx_module
from numpyro.handlers import do, seed, trace
from numpyro.infer import SVI, Predictive, Trace_ELBO
from numpyro.infer.autoguide import AutoNormal
from pydantic import BaseModel

numpyro.set_host_device_count(n=10)

rng_key = random.PRNGKey(seed=42)

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

%load_ext autoreload
%autoreload 2
%load_ext jaxtyping
%jaxtyping.typechecker beartype.beartype
%config InlineBackend.figure_format = "retina"

In [ ]:
class DataGenerationParameters(BaseModel):
    num_train: int
    num_test: int
    feature_dim: int
    z_gap: float
    y_gap: float


data_generation_parameters = DataGenerationParameters(
    num_train=10_000,
    num_test=2_000,
    feature_dim=10,
    z_gap=1.0,
    y_gap=3.0,
)

In [ ]:
def generate_data(
    num_data: int, feature_dim: int, z_gap: float, y_gap: float
) -> tuple[
    Float[Array, "num_data feature_dim"],
    Int[Array, " num_data"],
    Int[Array, " num_data"],
    Int[Array, " num_data"],
    Int[Array, " num_data"],
]:
    # Latent confounder z is binary.
    with numpyro.plate("num_data", num_data):
        z = numpyro.sample("z", dist.Bernoulli(0.5))

        # Covariates x are normally distributed, with higher variance for z=1.
        with numpyro.plate("feature_dim", feature_dim):
            x = numpyro.sample("x", dist.Normal(z * z_gap, 5 * z + 3 * (1 - z))).T

        # Treatment t is binary, with higher probability for z=1.
        t = numpyro.sample("t", dist.Bernoulli(0.75 * z + 0.25 * (1 - z)))

        # Outcome y is binary, with higher probability for z=1 and t=1.
        y = numpyro.sample("y", dist.Bernoulli(logits=y_gap * (z + 2 * (2 * t - 1))))

        # Compute true c-specific CATE for evaluation.
        t0_t1 = jnp.array([[0.0], [1.0]])
        y_t0_t1 = numpyro.sample(
            "y_t0_t1", dist.Bernoulli(logits=y_gap * (z + 2 * (2 * t0_t1 - 1)))
        )

        y_t0 = y_t0_t1[0]
        y_t1 = y_t0_t1[1]

        true_cates = y_t1 - y_t0

        return (x, t, y, z, true_cates)

In [ ]:
numpyro.render_model(
    generate_data,
    model_kwargs={
        "num_data": data_generation_parameters.num_train,
        "feature_dim": data_generation_parameters.feature_dim,
        "z_gap": data_generation_parameters.z_gap,
        "y_gap": data_generation_parameters.y_gap,
    },
)

In [ ]:
x_train: Float[Array, "num_train feature_dim"]
t_train: Int[Array, " num_train"]
y_train: Int[Array, " num_train"]

rng_key, rng_subkey = random.split(rng_key)

x_train, t_train, y_train, _, _ = trace(seed(generate_data, rng_subkey))(
    num_data=data_generation_parameters.num_train,
    feature_dim=data_generation_parameters.feature_dim,
    z_gap=data_generation_parameters.z_gap,
    y_gap=data_generation_parameters.y_gap,
)


x_test: Float[Array, "feature_dim num_test"]
z_test: Int[Array, " num_test"]
true_cates: Float[Array, " num_test"]

x_test, _, _, z_test, true_cates = trace(seed(generate_data, rng_subkey))(
    num_data=data_generation_parameters.num_test,
    feature_dim=data_generation_parameters.feature_dim,
    z_gap=data_generation_parameters.z_gap,
    y_gap=data_generation_parameters.y_gap,
)

In [ ]:
class FullyConnected(nnx.Module):
    def __init__(
        self, din: int, dout: int, hidden_layers: list[int], *, rngs: nnx.Rngs
    ) -> None:
        self.layers = nnx.List([])

        layer_dims = [din, *hidden_layers, dout]

        for in_dim, out_dim in pairwise(layer_dims):
            self.layers.append(nnx.Linear(in_dim, out_dim, rngs=rngs))

    def __call__(self, x: Array) -> Array:
        for layer in self.layers:
            x = jax.nn.elu(layer(x))
        return x


class DiagNormalNet(FullyConnected):
    def __init__(
        self, din: int, dout: int, hidden_layers: list[int], *, rngs: nnx.Rngs
    ):
        super().__init__(din=din, dout=2 * dout, hidden_layers=hidden_layers, rngs=rngs)

    def __call__(self, x: Array) -> tuple[Array, Array]:
        loc, scale = jnp.split(super().__call__(x), 2, axis=-1)
        return loc, jax.nn.softplus(scale)


class BernoulliNet(FullyConnected):
    def __init__(
        self, din: int, dout: int, hidden_layers: list[int], *, rngs: nnx.Rngs
    ):
        super().__init__(din, dout, hidden_layers, rngs=rngs)

    def __call__(self, x: Array) -> Array:
        return jnp.clip(super().__call__(x), a_min=-10, a_max=10)

In [ ]:
class ModelParameters(BaseModel):
    feature_dim: int
    hidden_dim: int
    latent_dim: int
    num_layers: int


model_parameters = ModelParameters(
    feature_dim=x_train.shape[1],
    latent_dim=5,
    hidden_dim=200,
    num_layers=3,
)

In [ ]:
rng_key, rng_subkey = random.split(rng_key)

x_nn_module = DiagNormalNet(
    din=model_parameters.latent_dim,
    dout=model_parameters.feature_dim,
    hidden_layers=[model_parameters.hidden_dim] * model_parameters.num_layers,
    rngs=nnx.Rngs(rng_subkey),
)

t_nn_module = BernoulliNet(
    din=model_parameters.latent_dim,
    dout=1,
    hidden_layers=[1],
    rngs=nnx.Rngs(rng_subkey),
)

y_nn_0_module = BernoulliNet(
    din=model_parameters.latent_dim,
    dout=1,
    hidden_layers=[model_parameters.hidden_dim] * model_parameters.num_layers,
    rngs=nnx.Rngs(rng_subkey),
)

y_nn_1_module = BernoulliNet(
    din=model_parameters.latent_dim,
    dout=1,
    hidden_layers=[model_parameters.hidden_dim] * model_parameters.num_layers,
    rngs=nnx.Rngs(rng_subkey),
)

In [ ]:
from jax._src.basearray import Array

rng_key, rng_subkey = random.split(rng_key)

x_nn_module = DiagNormalNet(
    din=model_parameters.latent_dim,
    dout=model_parameters.feature_dim,
    hidden_layers=[model_parameters.latent_dim]
    + [model_parameters.hidden_dim] * model_parameters.num_layers
    + [model_parameters.feature_dim],
    rngs=nnx.Rngs(rng_subkey),
)


def model(
    x: Float[Array, "num_data feature_dim"],
    t: Int[Array, " num_data"],
    y: Int[Array, " num_data"],
    latent_dim: int = 5,
) -> None:
    num_data, n_features = x.shape

    with numpyro.plate("latent_dim", latent_dim):
        z = numpyro.sample("z", dist.Normal(0, 1))

    x_nn = nnx_module("x_nn", x_nn_module)
    x_nn_z = x_nn(z)

    x_loc = x_nn_z[0][None, :]
    x_scale = x_nn_z[1][None, :]

    numpyro.sample(
        "x_obs",
        dist.Normal(x_loc, x_scale).expand([num_data, n_features]).to_event(2),
        obs=x,
    )

    t_nn = nnx_module("t_nn", t_nn_module)
    t_nn_z = t_nn(z)

    t_obs = numpyro.sample(
        "t_obs",
        dist.Bernoulli(logits=t_nn_z).expand([num_data]).to_event(1),
        obs=t,
    )

    y_nn_0 = nnx_module("y_nn_0", y_nn_0_module)
    y_nn_0_z = y_nn_0(z)

    y_nn_1 = nnx_module("y_nn_1", y_nn_1_module)
    y_nn_1_z = y_nn_1(z)

    y_logits = jnp.where(t_obs == 0, y_nn_0_z, y_nn_1_z)
    y_probs = numpyro.deterministic("y_probs", jax.nn.sigmoid(y_logits))

    numpyro.sample(
        "y_obs",
        dist.Bernoulli(probs=y_probs).expand([num_data]).to_event(1),
        obs=y,
    )


numpyro.render_model(
    model,
    model_kwargs={
        "x": x_train,
        "t": t_train,
        "y": y_train,
        "latent_dim": model_parameters.latent_dim,
    },
)

In [ ]:
%%time
guide = AutoNormal(model)

scheduler = optax.linear_onecycle_schedule(
    transition_steps=2_000,
    peak_value=0.008,
    pct_start=0.2,
    pct_final=0.8,
    div_factor=3,
    final_div_factor=4,
)

optimizer = optax.chain(
    optax.adam(learning_rate=scheduler),
    optax.contrib.reduce_on_plateau(
        factor=0.1,
        patience=10,
        accumulation_size=100,
    ),
)

svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
num_steps = 2_000

rng_key, rng_subkey = random.split(key=rng_key)

svi_result = svi.run(
    rng_subkey,
    num_steps,
    x=x_train,
    t=t_train,
    y=y_train,
    latent_dim=model_parameters.latent_dim,
)

fig, ax = plt.subplots(figsize=(9, 6))
ax.plot(svi_result.losses)
ax.set_title("ELBO loss", fontsize=18, fontweight="bold");

In [ ]:
params = svi_result.params

train_posterior_predictive = Predictive(
    model=model,
    guide=guide,
    params=params,
    num_samples=2_000,
)

# Generate samples for training data
rng_key, rng_subkey = random.split(key=rng_key)
train_posterior_predictive_samples = train_posterior_predictive(
    rng_subkey, x_train, t_train, y_train, model_parameters.latent_dim
)

# Convert to ArviZ InferenceData for analysis and visualization
train_idata = az.from_dict(
    posterior_predictive={
        # Add chain dimension for ArviZ compatibility
        k: np.expand_dims(a=np.asarray(v), axis=0)
        for k, v in train_posterior_predictive_samples.items()
    },
    coords={
        "feature": np.arange(x_train.shape[1]),
        "obs_idx": np.arange(x_train.shape[0]),
    },
    dims={
        "x_obs": ["obs_idx", "feature"],
        "t_obs": ["obs_idx"],
        "y_obs": ["obs_idx"],
    },
)


In [ ]:
do_0_model = do(model, {"t_obs": 0})
do_1_model = do(model, {"t_obs": 1})

rng_key, rng_subkey = random.split(rng_key)

train_do_0_model_samples = Predictive(
    do_0_model, guide=guide, params=params, num_samples=2_000, return_sites=["y_logits"]
)(
    rng_subkey,
    x_train,
    t_train,
    y_train,
    model_parameters.latent_dim,
)


train_do_1_model_samples = Predictive(
    do_1_model, guide=guide, params=params, num_samples=2_000, return_sites=["y_logits"]
)(
    rng_subkey,
    x_train,
    t_train,
    y_train,
    model_parameters.latent_dim,
)


train_do_0_idata = az.from_dict(
    posterior_predictive={
        # Add chain dimension for ArviZ compatibility
        k: np.expand_dims(a=np.asarray(v), axis=0)
        for k, v in train_do_0_model_samples.items()
    },
    coords={"obs_idx": np.arange(x_train.shape[0])},
    dims={"y_obs": ["obs_idx"]},
)

train_do_1_idata = az.from_dict(
    posterior_predictive={
        # Add chain dimension for ArviZ compatibility
        k: np.expand_dims(a=np.asarray(v), axis=0)
        for k, v in train_do_1_model_samples.items()
    },
    coords={"obs_idx": np.arange(x_train.shape[0])},
    dims={"y_probs": ["obs_idx"]},
)


In [ ]:
train_do_1_idata

In [ ]:
(
    train_do_1_idata["posterior_predictive"]["y_logits"]
    - train_do_0_idata["posterior_predictive"]["y_logits"]
).sum()